In [1]:
!pip install vega

     |████████████████████████████████| 1.1MB 23.5MB/s 
  Stored in directory: /root/.cache/pip/wheels/92/2e/85/0da206e5c19a13b02b1b5b27752bc9c304384d8b22e30dff0f
Successfully built vega


In [2]:
!pip install -U altair vega_datasets

Requirement already up-to-date: altair in /usr/local/lib/python3.6/dist-packages (3.1.0)
Requirement already up-to-date: vega_datasets in /usr/local/lib/python3.6/dist-packages (0.7.0)


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [5]:
import glob
import numpy as np
import librosa
import os
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import pandas as pd
import vega
import altair as alt
import pandas as pd
import scipy.signal
import scipy.fftpack as fft
from keras.layers import Input, Dense
from keras.models import Model

Using TensorFlow backend.


In [7]:
#---File reading---#
print('File reading...')
all_dirs = []
for root, dirs, files in os.walk('/content/gdrive/My Drive/Work and Research/MaSC Research/Coding/Dataset'):
        for name in files:
            if '.wav' in name:
                filedir = os.path.join(root, name)
                all_dirs.append(filedir)
file_no = len(all_dirs)
print('Number of files: ' + str(file_no))

#---Feature Computation---#
print('Feature Computation...')
all_mel = []
file_names = []
for i in range(file_no):
    if (librosa.get_duration(filename=all_dirs[i]) == 15.):
        file_names.append(all_dirs[i])
        
        #Progress report
        print(str(i) + '/' + str(file_no))
        if (i==file_no-1):
            print('---> Progress: 100%')
        elif (i==int(file_no*0.75)):
            print('---> Progress: 75%')
        elif (i==int(file_no*0.5)):
            print('---> Progress: 50%')
        elif (i==int(file_no*0.25)):
            print('---> Progress: 25%')

        #Load file
        y, sr = librosa.core.load(all_dirs[i])
        #Features
        mel = librosa.feature.melspectrogram(y=y, sr=sr)
        all_mel.append(mel.flatten())
        
print('Number of files with a duration of 15 seconds: ' + str(len(file_names)))

#---Principal Component Analysis---#
print('Principal Component Analysis...')
all_mel_pca5 = PCA(n_components=5).fit_transform(all_mel)
all_mel_pca10 = PCA(n_components=10).fit_transform(all_mel)
all_mel_pca20 = PCA(n_components=20).fit_transform(all_mel)
all_mel_pca50 = PCA(n_components=50).fit_transform(all_mel)

#---Component Standardization---#
print('Component Standardization...')
scl1 = StandardScaler()
all_mel_pca5_scaled = scl1.fit_transform(all_mel_pca5)
scl2 = StandardScaler()
all_mel_pca10_scaled = scl1.fit_transform(all_mel_pca10)
scl3 = StandardScaler()
all_mel_pca20_scaled = scl1.fit_transform(all_mel_pca20)
scl4 = StandardScaler()
all_mel_pca50_scaled = scl1.fit_transform(all_mel_pca50)


#---t-SNE---#
print('t-SNE...')
all_mel_pca5_scaled_red2 = TSNE(n_components=2).fit_transform(all_mel_pca5_scaled)
all_mel_pca10_scaled_red2 = TSNE(n_components=2).fit_transform(all_mel_pca10_scaled)
all_mel_pca20_scaled_red2 = TSNE(n_components=2).fit_transform(all_mel_pca20_scaled)
all_mel_pca50_scaled_red2 = TSNE(n_components=2).fit_transform(all_mel_pca50_scaled)

#---KMeans---#
print('KMeans...')
kmeans1 = KMeans(n_clusters=5, random_state=42).fit(all_mel_pca5_scaled_red2)
clusters1 = kmeans1.predict(all_mel_pca5_scaled_red2)
kmeans2 = KMeans(n_clusters=5, random_state=42).fit(all_mel_pca10_scaled_red2)
clusters2 = kmeans2.predict(all_mel_pca10_scaled_red2)
kmeans3 = KMeans(n_clusters=5, random_state=42).fit(all_mel_pca20_scaled_red2)
clusters3 = kmeans3.predict(all_mel_pca20_scaled_red2)
kmeans4 = KMeans(n_clusters=5, random_state=42).fit(all_mel_pca50_scaled_red2)
clusters4 = kmeans4.predict(all_mel_pca50_scaled_red2)

#---Intensity---#
print('Intensity...')
all_db = []
for i in range(file_no):
    if (librosa.get_duration(filename=all_dirs[i]) == 15.):
 
        #Progress report
        if (i==file_no-1):
            print('---> Progress: 100%')
        elif (i==int(file_no*0.75)):
            print('---> Progress: 75%')
        elif (i==int(file_no*0.5)):
            print('---> Progress: 50%')
        elif (i==int(file_no*0.25)):
            print('---> Progress: 25%')

        #Load file
        y, sr = librosa.core.load(all_dirs[i])
        #Features
        stft = librosa.core.stft(y=y)
        db = librosa.core.power_to_db(stft)
        all_db.append(np.mean(db)) 

#---Intensity Standardization---#
print('Intensity Standardization...')
scl5 = StandardScaler()
all_db_scaled = scl1.fit_transform((np.asarray(all_db)).reshape(-1, 1))
all_db_scaled = all_db_scaled.reshape(1,-1)

#---Scatter---#
print('Preparing dataframe...')
xmel5 = []
ymel5 = []
for i in range(len(all_mel_pca5_scaled_red2)):
    xmel5.append(all_mel_pca5_scaled_red2[i][0])
    ymel5.append(all_mel_pca5_scaled_red2[i][1])
xmel10 = []
ymel10 = []
for i in range(len(all_mel_pca10_scaled_red2)):
    xmel10.append(all_mel_pca10_scaled_red2[i][0])
    ymel10.append(all_mel_pca10_scaled_red2[i][1])
xmel20 = []
ymel20 = []
for i in range(len(all_mel_pca20_scaled_red2)):
    xmel20.append(all_mel_pca20_scaled_red2[i][0])
    ymel20.append(all_mel_pca20_scaled_red2[i][1])
xmel50 = []
ymel50 = []
for i in range(len(all_mel_pca50_scaled_red2)):
    xmel50.append(all_mel_pca50_scaled_red2[i][0])
    ymel50.append(all_mel_pca50_scaled_red2[i][1])

#---Dataframes---#
df11 = pd.DataFrame({'x': xmel5, 'y': ymel5, 'color': clusters1, 'path': np.asarray(file_names), 'filename': np.asarray(file_names)})
df12 = pd.DataFrame({'x': xmel5, 'y': ymel5, 'color': all_db_scaled[0], 'path': np.asarray(file_names), 'filename': np.asarray(file_names)})

df21 = pd.DataFrame({'x': xmel10, 'y': ymel10, 'color': clusters2, 'path': np.asarray(file_names), 'filename': np.asarray(file_names)})
df22 = pd.DataFrame({'x': xmel10, 'y': ymel10, 'color': all_db_scaled[0], 'path': np.asarray(file_names), 'filename': np.asarray(file_names)})

df31 = pd.DataFrame({'x': xmel20, 'y': ymel20, 'color': clusters3, 'path': np.asarray(file_names), 'filename': np.asarray(file_names)})
df32 = pd.DataFrame({'x': xmel20, 'y': ymel20, 'color': all_db_scaled[0], 'path': np.asarray(file_names), 'filename': np.asarray(file_names)})

df41 = pd.DataFrame({'x': xmel50, 'y': ymel50, 'color': clusters4, 'path': np.asarray(file_names), 'filename': np.asarray(file_names)})
df42 = pd.DataFrame({'x': xmel50, 'y': ymel50, 'color': all_db_scaled[0], 'path': np.asarray(file_names), 'filename': np.asarray(file_names)})

#---Visualization declerations---#
print('Visualization...')
#Clusters
chart11 = alt.Chart(df11).mark_circle(opacity=0.6, size=60).encode(x='x', y='y', color='color:N', href='path', tooltip=['filename']).interactive()
#Intensity
chart12 = alt.Chart(df12).mark_circle(opacity=0.6, size=60).encode(x='x', y='y', color='color:O', href='path', tooltip=['filename']).interactive()
#Overlap Clusters
chart13 = alt.Chart(df11).mark_circle(size=100).encode(x='x', y='y', color='color:N', href='path', tooltip=['filename']).interactive()
#Overlap Intensity
chart14 = alt.Chart(df12).mark_circle(size=60).encode(x='x', y='y', color='color:O', href='path', tooltip=['filename']).interactive()

#Clusters
chart21 = alt.Chart(df21).mark_circle(opacity=0.6, size=60).encode(x='x', y='y', color='color:N', href='path', tooltip=['filename']).interactive()
#Intensity
chart22 = alt.Chart(df22).mark_circle(opacity=0.6, size=60).encode(x='x', y='y', color='color:O', href='path', tooltip=['filename']).interactive()
#Overlap Clusters
chart23 = alt.Chart(df21).mark_circle(size=100).encode(x='x', y='y', color='color:N', href='path', tooltip=['filename']).interactive()
#Overlap Intensity
chart24 = alt.Chart(df22).mark_circle(size=60).encode(x='x', y='y', color='color:O', href='path', tooltip=['filename']).interactive()

#Clusters
chart31 = alt.Chart(df31).mark_circle(opacity=0.6, size=60).encode(x='x', y='y', color='color:N', href='path', tooltip=['filename']).interactive()
#Intensity
chart32 = alt.Chart(df32).mark_circle(opacity=0.6, size=60).encode(x='x', y='y', color='color:O', href='path', tooltip=['filename']).interactive()
#Overlap Clusters
chart33 = alt.Chart(df31).mark_circle(size=100).encode(x='x', y='y', color='color:N', href='path', tooltip=['filename']).interactive()
#Overlap Intensity
chart34 = alt.Chart(df32).mark_circle(size=60).encode(x='x', y='y', color='color:O', href='path', tooltip=['filename']).interactive()

#Clusters
chart41 = alt.Chart(df41).mark_circle(opacity=0.6, size=60).encode(x='x', y='y', color='color:N', href='path', tooltip=['filename']).interactive()
#Intensity
chart42 = alt.Chart(df42).mark_circle(opacity=0.6, size=60).encode(x='x', y='y', color='color:O', href='path', tooltip=['filename']).interactive()
#Overlap Clusters
chart43 = alt.Chart(df41).mark_circle(size=100).encode(x='x', y='y', color='color:N', href='path', tooltip=['filename']).interactive()
#Overlap Intensity
chart44 = alt.Chart(df42).mark_circle(size=60).encode(x='x', y='y', color='color:O', href='path', tooltip=['filename']).interactive()

#---Display---#
chart11
chart12
chart13 + chart14

chart21
chart22
chart23 + chart24

chart31
chart32
chart33 + chart34

chart41
chart42
chart43 + chart44

File reading...
Number of files: 2332
Feature Computation...
0/2332
1/2332
2/2332
3/2332
4/2332
5/2332
6/2332
7/2332
8/2332
9/2332
10/2332
11/2332
12/2332
13/2332
14/2332
15/2332
16/2332
17/2332
18/2332
19/2332
20/2332
21/2332
22/2332
23/2332
24/2332
25/2332
26/2332
27/2332
28/2332
29/2332
30/2332
31/2332
32/2332
33/2332
34/2332
35/2332
36/2332
37/2332
38/2332
39/2332
40/2332
41/2332
42/2332
43/2332
44/2332
45/2332
46/2332
47/2332
48/2332
49/2332
50/2332
51/2332
52/2332
53/2332
54/2332
55/2332
56/2332
57/2332
58/2332
59/2332
60/2332
61/2332
62/2332
63/2332
64/2332
65/2332
66/2332
67/2332
68/2332
69/2332
70/2332
71/2332
72/2332
73/2332
74/2332
75/2332
76/2332
77/2332
78/2332
79/2332
80/2332
81/2332
82/2332
83/2332
84/2332
85/2332
86/2332
87/2332
88/2332
89/2332
90/2332
91/2332
92/2332
93/2332
94/2332
95/2332
96/2332
97/2332
98/2332
99/2332
100/2332
101/2332
102/2332
103/2332
104/2332
105/2332
106/2332
107/2332
108/2332
109/2332
110/2332
111/2332
112/2332
113/2332
114/2332
115/2332
116/2

/usr/local/lib/python3.6/dist-packages/librosa/core/spectrum.py:888: UserWarning: power_to_db was called on complex input so phase information will be discarded. To suppress this warning, call power_to_db(np.abs(D)**2) instead.
  warnings.warn('power_to_db was called on complex input so phase '
/usr/local/lib/python3.6/dist-packages/librosa/core/spectrum.py:888: UserWarning: power_to_db was called on complex input so phase information will be discarded. To suppress this warning, call power_to_db(np.abs(D)**2) instead.
  warnings.warn('power_to_db was called on complex input so phase '
/usr/local/lib/python3.6/dist-packages/librosa/core/spectrum.py:888: UserWarning: power_to_db was called on complex input so phase information will be discarded. To suppress this warning, call power_to_db(np.abs(D)**2) instead.
  warnings.warn('power_to_db was called on complex input so phase '
/usr/local/lib/python3.6/dist-packages/librosa/core/spectrum.py:888: UserWarning: power_to_db was called on comp

---> Progress: 25%


/usr/local/lib/python3.6/dist-packages/librosa/core/spectrum.py:888: UserWarning: power_to_db was called on complex input so phase information will be discarded. To suppress this warning, call power_to_db(np.abs(D)**2) instead.
  warnings.warn('power_to_db was called on complex input so phase '
/usr/local/lib/python3.6/dist-packages/librosa/core/spectrum.py:888: UserWarning: power_to_db was called on complex input so phase information will be discarded. To suppress this warning, call power_to_db(np.abs(D)**2) instead.
  warnings.warn('power_to_db was called on complex input so phase '
/usr/local/lib/python3.6/dist-packages/librosa/core/spectrum.py:888: UserWarning: power_to_db was called on complex input so phase information will be discarded. To suppress this warning, call power_to_db(np.abs(D)**2) instead.
  warnings.warn('power_to_db was called on complex input so phase '
/usr/local/lib/python3.6/dist-packages/librosa/core/spectrum.py:888: UserWarning: power_to_db was called on comp

---> Progress: 50%


/usr/local/lib/python3.6/dist-packages/librosa/core/spectrum.py:888: UserWarning: power_to_db was called on complex input so phase information will be discarded. To suppress this warning, call power_to_db(np.abs(D)**2) instead.
  warnings.warn('power_to_db was called on complex input so phase '
/usr/local/lib/python3.6/dist-packages/librosa/core/spectrum.py:888: UserWarning: power_to_db was called on complex input so phase information will be discarded. To suppress this warning, call power_to_db(np.abs(D)**2) instead.
  warnings.warn('power_to_db was called on complex input so phase '
/usr/local/lib/python3.6/dist-packages/librosa/core/spectrum.py:888: UserWarning: power_to_db was called on complex input so phase information will be discarded. To suppress this warning, call power_to_db(np.abs(D)**2) instead.
  warnings.warn('power_to_db was called on complex input so phase '
/usr/local/lib/python3.6/dist-packages/librosa/core/spectrum.py:888: UserWarning: power_to_db was called on comp

---> Progress: 75%


/usr/local/lib/python3.6/dist-packages/librosa/core/spectrum.py:888: UserWarning: power_to_db was called on complex input so phase information will be discarded. To suppress this warning, call power_to_db(np.abs(D)**2) instead.
  warnings.warn('power_to_db was called on complex input so phase '
/usr/local/lib/python3.6/dist-packages/librosa/core/spectrum.py:888: UserWarning: power_to_db was called on complex input so phase information will be discarded. To suppress this warning, call power_to_db(np.abs(D)**2) instead.
  warnings.warn('power_to_db was called on complex input so phase '
/usr/local/lib/python3.6/dist-packages/librosa/core/spectrum.py:888: UserWarning: power_to_db was called on complex input so phase information will be discarded. To suppress this warning, call power_to_db(np.abs(D)**2) instead.
  warnings.warn('power_to_db was called on complex input so phase '
/usr/local/lib/python3.6/dist-packages/librosa/core/spectrum.py:888: UserWarning: power_to_db was called on comp

OSError: ignored